In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import fftpack
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from scipy.optimize import leastsq
from scipy import optimize
import sys
from sympy.solvers import solve
from sympy import Symbol
import pandas as pd
from SimulationLoop_Analysis import simulation_signal_analysis

In [1]:
Optimize_Param_Vertical = []
Optimize_Param_East     = []
Optimize_Param_North    = []
Optimize_Param_West     = []
Optimize_Param_South    = []
counter = 0
obj = simulation_signal_analysis()
beam_direction = ["Vertical_Beam","East_Beam","North_Beam","West_Beam","South_Beam"]
for beam in beam_direction:
    counter += 1
    directory = glob.glob("../Signal/Simulation_loops/"+beam+"/*")
    directory = sorted(directory)
    for dirs in directory:
        if (counter == 1):
            Optimize_Param_Vertical.append(obj.file_analysis(dirs+"/*"))
        elif(counter == 2):
            Optimize_Param_East.append(obj.file_analysis(dirs+"/*"))
        elif(counter == 3):
            Optimize_Param_North.append(obj.file_analysis(dirs+"/*"))
        elif(counter == 4):
            Optimize_Param_West.append(obj.file_analysis(dirs+"/*"))
        elif(counter == 5):
            Optimize_Param_South.append(obj.file_analysis(dirs+"/*"))
        else:
            print("Error!There is only 5 beams!")

In [10]:
Optimize_Param_Vertical = np.array(Optimize_Param_Vertical)
Optimize_Param_East     = np.array(Optimize_Param_East)
Optimize_Param_North    = np.array(Optimize_Param_North)
Optimize_Param_West     = np.array(Optimize_Param_West)
Optimize_Param_South    = np.array(Optimize_Param_South)

In [12]:
# Monostatic Calculation
estimated_u = -(Optimize_Param_East-Optimize_Param_West)/(2*np.sin(10*np.pi/180))
estimated_v = -(Optimize_Param_North-Optimize_Param_South)/(2*np.sin(10*np.pi/180))
estimated_w = -Optimize_Param_Vertical
res_mono = np.vstack((estimated_u,estimated_v,estimated_w))
print(res_mono)

[[  9.45403423  10.03158239   9.36163443   9.27188447   9.44597512
    9.40345708  10.02065666  10.14770965   9.42509982   9.35789588
    9.29899293   9.03183371   9.61247331   9.12902107   9.28528891
    9.56010653   9.42905772   9.19452256   9.30442469  10.01324534
   10.53737536   9.22758601  10.12431187   9.96338997   9.17175529
    9.27430018   9.38906585   8.82098406   9.86700294   8.9086056
    9.06766761   9.65547547   9.71144583   9.81276498   9.92860246
   10.18645908   9.68815776  10.02787108  10.32543319  10.00970691
    8.71838454   9.11999728   9.47006498  10.2299265    8.99966357
    9.66045493   9.66895789   9.06918634   9.94852741   9.59767708
    9.75560645   8.72356545   8.2110131    8.93635489   9.51855012
    9.512355     9.67748583   9.69162503  10.13243472   9.11649587
   10.45924637   9.43868101  10.94404457   9.00644962  10.14141967
    8.34232984   9.53265519   9.38073948   9.98533134   8.72206893
    9.93920337   9.94273625   9.77464979   9.52624481  10.58251

In [13]:
# Bistatic Calculation

# The calculation of the range R(target)
def pseudo_get(height):
    def Range_Calcu(theta):
        R = Symbol('R')
        radian = theta*np.pi/180
        Result_R = solve((height*2-R)**2-100**2-R**2+200*R*np.cos(radian),R)# 600是说来回距离加一起
        return float(Result_R[0])

    TX_Coordi = np.array([0,0,0])
    RX_Coordi = np.array([100,0,0])
    Angle_Para_1 = np.sin(80*np.pi/180)
    Angle_Para_2 = np.cos(80*np.pi/180)

    def Normal_Vector(Para_X,Para_Y,Para_Z): # The calculation of normalized vector
        Target_Coordi=np.array([Para_X,Para_Y,Para_Z])
        Unnormal_Vector_TX = Target_Coordi-TX_Coordi
        Unnormal_Vector_RX = Target_Coordi-RX_Coordi
        Unit_Vector_TX=1/np.linalg.norm(Unnormal_Vector_TX) * Unnormal_Vector_TX
        Unit_Vector_RX=1/np.linalg.norm(Unnormal_Vector_RX) * Unnormal_Vector_RX
        Unit_Vector = -(Unit_Vector_TX+Unit_Vector_RX)*1/2 # マイナスを取る意味：近づいてくる方向が正
        return Unit_Vector
    # Vertical Beam
    Para_VTC_Z = Range_Calcu(90)
    Vertical_Unit_Vector=Normal_Vector(0,0,Para_VTC_Z) # Vertical Target Coordinates

    # East Beam
    Para_ETC_X=Range_Calcu(80)*Angle_Para_2
    Para_ETC_Z=Range_Calcu(80)*Angle_Para_1
    East_Unit_Vector=Normal_Vector(Para_ETC_X,0,Para_ETC_Z)

    # North Beam
    Para_NTC=Range_Calcu(90)
    Para_NTC_Y=Para_NTC*Angle_Para_2
    Para_NTC_Z=Para_NTC*Angle_Para_1
    North_Unit_Vector=Normal_Vector(0,Para_NTC_Y,Para_NTC_Z)
    # West Beam
    Para_WTC_X=-Range_Calcu(100)*Angle_Para_2
    Para_WTC_Z=Range_Calcu(100)*Angle_Para_1
    West_Unit_Vector=Normal_Vector(Para_WTC_X,0,Para_WTC_Z)

    # South Beam
    Para_STC=Range_Calcu(90)
    Para_STC_Y=-Para_STC*Angle_Para_2
    Para_STC_Z=Para_STC*Angle_Para_1
    South_Unit_Vector=Normal_Vector(0,Para_STC_Y,Para_STC_Z)
    # 擬似逆行列の計算
    Original_Mat=np.vstack((Vertical_Unit_Vector,East_Unit_Vector,North_Unit_Vector,West_Unit_Vector,South_Unit_Vector))
    Pseudo_Inv = np.linalg.pinv(Original_Mat)
    return Pseudo_Inv

In [14]:
Pseudo_Inv=pseudo_get(300)

In [15]:
#ドップラ速度の導入
Doppler_V = np.array([Optimize_Param_Vertical,
                      Optimize_Param_East,
                      Optimize_Param_North,
                      Optimize_Param_West,
                      Optimize_Param_South])
# バイスタティックの計算
Bi_Result=np.matmul(Pseudo_Inv,Doppler_V)
print(Bi_Result)

[[  9.82264023  10.412853     9.71595764   9.62710155   9.81336263
    9.74964686  10.37256018  10.51775077   9.76391874   9.71323318
    9.628639     9.34749537   9.98062942   9.49142233   9.62689907
    9.93382931   9.77576312   9.53960348   9.64559879  10.38379811
   10.91921756   9.60354619  10.48748688  10.34288032   9.52345953
    9.62150541   9.74955139   9.1702188   10.23800812   9.25655756
    9.39960701  10.006371    10.10429064  10.19497516  10.26719154
   10.57160624  10.05140957  10.41078849  10.73227532  10.38555405
    9.06096319   9.45944907   9.82092229  10.6189738    9.35178539
   10.02530801  10.02237898   9.39001941  10.35932667   9.91997034
   10.12678087   9.0631531    8.54158229   9.26154872   9.88966527
    9.86569705  10.02750049  10.02827903  10.49443632   9.49105924
   10.86612023   9.80383574  11.35376887   9.37181742  10.52647515
    8.67266278   9.89864079   9.75148145  10.37199833   9.05180595
   10.30665461  10.31434624  10.13035687   9.8701336   10.9853